In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers
from time import sleep
import pickle

In [4]:
from drivers.KeysightAWG import *
from drivers.Agilent_PNA_L import *

In [16]:
from scipy.optimize import minimize

In [ ]:
rc("font", size=15)

## DC offset calibration

In [ ]:
pna_l.set_nop(100)
pna_l.set_xlim(4.5e9, 9e9)
pna_l.set_bandwidth(1000)

In [ ]:
plt.plot(*get_vna_trace(pna_l))
awg.apply_waveform(WaveformType.dc,0,0,1,1)
plt.plot(*get_vna_trace(pna_l))
awg.apply_waveform(WaveformType.dc,0,0,0,1)
# plt.plot(*get_vna_trace(pna_l))
# awg.set_outp2(0)
# awg.set_outp2_compl(0)
# awg.set_outp2(1)
# awg.apply_waveform(WaveformType.dc,0,0,1,2)
# plt.plot(*get_vna_trace(pna_l))

In [ ]:
def get_transmission_data(N=101):
    transmission_data = []
    V_vals =  linspace(-0.1,0.1,N)
    for V_dc1 in V_vals:
        for V_dc2 in V_vals:
            print("\r", V_dc1, V_dc2, end="", flush=True)
            awg.apply_waveform(WaveformType.dc,0,0,V_dc1,1)
            awg.apply_waveform(WaveformType.dc,0,0,V_dc2,2)
            transmission_data.append(get_vna_trace(pna_l)[1])
    return V_vals, array(transmission_data)

In [ ]:
V_vals, transmission_data=get_transmission_data(201)

In [ ]:
with open("marki_iq_4509_cal_data.pkl", "wb") as f:
    pickle.dump((V_vals, transmission_data.reshape(V_vals.shape*2+(100,)), 2), f)

In [ ]:
# pcolormesh(V_vals, V_vals, 20*log10(np.min(transmission_data.reshape(V_vals.shape*2+(pna_l.get_nop(),)), 2)), cmap="RdBu_r")
# pcolormesh(V_vals, V_vals, 20*log10(transmission_data.reshape(V_vals.shape*2+(5,)))[:,:,3], cmap="RdBu_r")
pcolormesh(V_vals, V_vals, 20*log10(transmission_data.reshape(V_vals.shape*2+(100,)))[:,:,10], cmap="RdBu_r")

# plt.xlim(-5,5)
# plt.ylim(-5,5)
plt.axis("tight")
plt.gcf().set_size_inches(12,6)
cb = plt.colorbar()
plt.grid()
cb.set_label("$|S_{21}|$", fontsize=18)

In [ ]:
data = 20*log10(transmission_data.reshape(V_vals.shape*2+(100,)))[:,:,10]

In [ ]:
x,y = np.unravel_index(np.argmin(data),data.shape)
V_vals[x], V_vals[y]

## Full calibration with EXA spectrum analyzer

### DC with no pulse (DC$^0$)

In [9]:
reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import E8257D

In [10]:
def loss_function(voltages):
    vdc1, vdc2 = voltages
    awg.output_continuous_wave(frequency=100e6, amplitude=0, phase=0, offset=vdc1, channel=1)
    awg.output_continuous_wave(frequency=100e6, amplitude=0, phase=0, offset=vdc2, channel=2)
    exa.prepare_for_stb()
    exa.sweep_single()
    exa.wait_for_stb()
    answer = exa.get_tracedata()
    exa.set_averages(1+int(exp((-answer-51)/10)))
    print("\r", voltages, answer, end=", ", flush=True)
    return answer

In [12]:
exa = Agilent_EXA_N9010A("SA", "EXA")
awg = KeysightAWG("AWG2")
mwsrc = E8257D("MXG")

In [13]:
mwsrc.set_frequency(5e9)
mwsrc.set_power(10)

In [14]:
exa.set_nop(1)
exa.set_span(0e6)
exa.set_bandwidth(50)

True

In [18]:
%%time
res = minimize(loss_function, (.1,.1), method="Nelder-Mead", options={"disp":True, "maxiter":50, "xatol":0.1, "fatol":10})
print(res)

 [-0.006411   -0.01100238] [-64.59459686], Warning: Maximum number of iterations has been exceeded.
  status: 2
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 105
 success: False
     nit: 50
     fun: -71.87890625
       x: array([-0.00641219, -0.01100209])
Wall time: 1min 1s


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: OptimizeWarning: Unknown solver options: fatol, xatol
  if __name__ == '__main__':


In [ ]:
loss_function(res.x)

In [ ]:
exa.sweep_single()

### Continuous IF wave calibration

In [ ]:
f_lo = 5e9
f_if = 1e8

In [ ]:
def loss_function_if_dc(voltages, args):
    vdc1, vdc2 = voltages
    amp1, amp2 = args[0]
    phase = args[1]
    awg.output_continuous_wave(frequency=100e6, amplitude=amp1, phase=phase, offset=vdc1, channel=1)
    awg.output_continuous_wave(frequency=100e6, amplitude=amp2, phase=0, offset=vdc2, channel=2)
    exa.prepare_for_stb();exa.sweep_single();exa.wait_for_stb()
    data = exa.get_tracedata()
    answer =  data[1]
    print("\rDC offsets: ", voltages, data, end=", ", flush=True)
    return answer
def loss_function_if_amps(amplitudes, args):
    amp1, amp2 = amplitudes
    vdc1, vdc2 = args[0]
    phase = args[1]
    awg.output_continuous_wave(frequency=100e6, amplitude=amp1, phase=phase, offset=vdc1, channel=1)
    awg.output_continuous_wave(frequency=100e6, amplitude=amp2, phase=0, offset=vdc2, channel=2)
    exa.prepare_for_stb();exa.sweep_single();exa.wait_for_stb()
    data = exa.get_tracedata()
    answer =  data[2] + abs(-15 - data[0])*10
    print("\rAmplitudes: ", amplitudes, data, end=", ", flush=True)
    return answer
def loss_function_if_phase(phase, args):
    vdc1, vdc2 = args[0]
    amp1, amp2 = args[1]
    awg.output_continuous_wave(frequency=100e6, amplitude=amp1, phase=phase, offset=vdc1, channel=1)
    awg.output_continuous_wave(frequency=100e6, amplitude=amp2, phase=0, offset=vdc2, channel=2)
    exa.prepare_for_stb();exa.sweep_single();exa.wait_for_stb()
    data = exa.get_tracedata()
    answer =  data[2] - data[0]
    print("\rPhase: ", phase/pi, data, end=", ", flush=True)
    return answer

In [ ]:
def iterate_minimization(prev_results, n=2):
    exa.setup_list_sweep([f_lo-f_if, f_lo, f_lo+f_if], [200]*3)
    res_dc = minimize(loss_function_if_dc, prev_results["if_dc"], args=[prev_results["if_amps"], prev_results["if_phase"]],
                      method="Nelder-Mead", options={"maxiter":30})
    res_amps = minimize(loss_function_if_amps, prev_results["if_amps"], args=[res_dc.x, prev_results["if_phase"]],
                        method="Nelder-Mead", options={"maxiter":20})
    res_phase = minimize(loss_function_if_phase, prev_results["if_phase"], args=[res_dc.x, res_amps.x],
                         method="Nelder-Mead", options={"maxiter":20})
    results["if_dc"] = res_dc.x
    results["if_amps"] = res_amps.x
    results["if_phase"] = res_phase.x
    if(n-1==0):
        return
    iterate_minimization(results, n-1)

In [ ]:
results = {"if_dc":(0.1, 0.1), "if_amps":(0.5,0.5), "if_phase":-pi*0.54}

In [ ]:
iterate_minimization(results)

In [ ]:
exa.prepare_for_stb();exa.sweep_single();exa.wait_for_stb()
exa.get_tracedata()

Check results

In [ ]:
exa.setup_swept_sa(4.9e9, 800e6, nop=501, rbw=10e3)
exa.prepare_for_stb()
exa.sweep_single()
exa.wait_for_stb()

In [ ]:
plt.plot(exa.get_freqpoints(), exa.get_tracedata())
plt.grid()
plt.gcf().set_size_inches(15,5)

In [ ]:
try:
    with open("LOL") as f:
        pass
except FileNotFoundError:
    print("LOL")